# FIFA2019

## 用 Python CSV 套件，處理資料

In [1]:
fifa_source = './data/lite_fifa19.csv'

In [2]:
%%time 

import datetime

import  csv

fifa_source = './data/lite_fifa19.csv'

# file handle 

with open(fifa_source, 'r', newline='') as f:
    rows = csv.reader(f, delimiter=',')
    # rows is a iterable , next() and for..looping
    
    header = next(rows)
    lines = []
    for each_row in rows:
        lines.append(each_row)

print('done')

done
CPU times: user 110 ms, sys: 22.2 ms, total: 132 ms
Wall time: 138 ms


In [3]:
len(header), len(lines), len(lines[0])

(31, 18207, 31)

In [6]:
%%time

def create_dict_from_line(hd, line):
    output = {}
    for k, v in zip(hd, line):
        output[k] = v
    return output

fifa = [create_dict_from_line(header, line) for line in lines]

CPU times: user 62.2 ms, sys: 14.3 ms, total: 76.5 ms
Wall time: 78.2 ms


In [23]:
fifa[0]

{'Name': 'L. Messi',
 'Age': '31',
 'Nationality': 'Argentina',
 'Overall': '94',
 'Potential': '94',
 'Club': 'FC Barcelona',
 'Value': '110.5',
 'Wage': '565.0',
 'Height': "5'7",
 'Weight': '159lbs',
 'Special': '2202',
 'Preferred Foot': 'Left',
 'Weak Foot': '4.0',
 'Acceleration': '91.0',
 'SprintSpeed': '86.0',
 'Agility': '91.0',
 'Balance': '95.0',
 'BallControl': '96.0',
 'Composure': '96.0',
 'Dribbling': '97.0',
 'Reactions': '95.0',
 'Skill Moves': '4.0',
 'Work Rate': 'Medium/ Medium',
 'Body Type': 'Messi',
 'Position': 'RF',
 'ShotPower': '85.0',
 'Jumping': '68.0',
 'LongShots': '94.0',
 'Penalties': '75.0',
 'Jersey Number': '10.0'}

## our toolbox

In [ ]:
from collections import Counter 

# 一個欄位有多少相異值

def find_unique_values(key, data):
    return list(set([ath[key] for ath in data]))

# 符合指定條件的運動員有多少個： 國籍是紐西蘭的運動員有幾個？
def find_specific(key, val, data):
    return [ath['Name'] for ath in fifa if ath[key] == val]

# find_specific('Nationality', 'Dominican Republic', fifa)

# 出現個數的排行榜
def find_top_n(key, data, n):
    a = [ath[key] for ath in data]  # item_id_list, count_item_id_list
    return [c[0] for c in Counter(a).most_common()[:n]]

In [20]:
nationality = find_unique_values('Nationality', fifa)
clubs = find_unique_values('Club', fifa)

## Q1: 那個球會的球員平均薪水最高

In [27]:
def calc_club_wages(c):
    wage_raw = [float(ath['Wage']) for ath in fifa if ath['Club'] == c]
    wage_avg = sum(wage_raw)/len(wage_raw)
    return (wage_avg, c)

wage_list = [calc_club_wages(c) for c in clubs]
sorted(wage_list, reverse=True)[:10]

[(152.03030303030303, 'Real Madrid'),
 (146.57575757575756, 'FC Barcelona'),
 (131.68, 'Juventus'),
 (113.36363636363636, 'Manchester City'),
 (102.75757575757575, 'Manchester United'),
 (98.45454545454545, 'Chelsea'),
 (87.93939393939394, 'Liverpool'),
 (79.48484848484848, 'Tottenham Hotspur'),
 (78.82758620689656, 'FC Bayern München'),
 (78.42424242424242, 'Arsenal')]

## Q2 那個國家（地區）球員的平均薪水最高

In [39]:
def calc_nation_wages(n):
    wage_raw = [ float(ath['Wage']) for ath in fifa if ath['Nationality'] == n]
    wage_avg = sum(wage_raw)/len(wage_raw)
    return (wage_avg, n)

top_n = find_top_n('Nationality', fifa, 30)

wage_list = [
    calc_nation_wages(nation)
    for nation in nationality
    if nation in top_n
]

sorted(wage_list, reverse=True)[:10]

[(18.48076923076923, 'Belgium'),
 (17.778718258766627, 'Brazil'),
 (17.006711409395972, 'Uruguay'),
 (16.32276119402985, 'Spain'),
 (14.361538461538462, 'Senegal'),
 (14.31055900621118, 'Portugal'),
 (14.053610503282275, 'France'),
 (13.478632478632479, 'Italy'),
 (11.502668089647813, 'Argentina'),
 (10.442244224422442, 'Turkey')]

## Q3 球衣背號與薪水

In [37]:
float('')

ValueError: could not convert string to float: ''

In [42]:
def num_conv(s):
    try:
        return int(float(s))
    except ValueError:
        return 0  # Null , NaN 
    
jersey_list = [
    (float(ath['Wage']), ath['Name'])
     for ath in fifa
    # filter rule <- jersey number
    if num_conv(ath['Jersey Number']) == 7
]

sorted(jersey_list, reverse=True)[:5]

[(405.0, 'Cristiano Ronaldo'),
 (355.0, 'K. De Bruyne'),
 (340.0, 'Coutinho'),
 (215.0, 'A. Sánchez'),
 (145.0, 'H. Mkhitaryan')]

# Pandas Way 

In [1]:
import pandas as pd

fifa_source = './data/lite_fifa19.csv'

df = pd.read_csv(fifa_source)

df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Height,Weight,...,Skill Moves,Work Rate,Body Type,Position,ShotPower,Jumping,LongShots,Dribbling.1,Penalties,Jersey Number
0,L. Messi,31,Argentina,94,94,FC Barcelona,110.5,565.0,5'7,159lbs,...,4.0,Medium/ Medium,Messi,RF,85.0,68.0,94.0,97.0,75.0,10.0
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77.0,405.0,6'2,183lbs,...,5.0,High/ Low,C. Ronaldo,ST,95.0,95.0,93.0,88.0,85.0,7.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118.5,290.0,5'9,150lbs,...,5.0,High/ Medium,Neymar,LW,80.0,61.0,82.0,96.0,81.0,10.0
3,De Gea,27,Spain,91,93,Manchester United,72.0,260.0,6'4,168lbs,...,1.0,Medium/ Medium,Lean,GK,31.0,67.0,12.0,18.0,40.0,1.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102.0,355.0,5'11,154lbs,...,4.0,High/ High,Normal,RCM,91.0,63.0,91.0,86.0,79.0,7.0


## 資料集基本資料

In [2]:
df.shape

(18207, 31)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            18207 non-null  object 
 1   Age             18207 non-null  int64  
 2   Nationality     18207 non-null  object 
 3   Overall         18207 non-null  int64  
 4   Potential       18207 non-null  int64  
 5   Club            17966 non-null  object 
 6   Value           18207 non-null  float64
 7   Wage            18207 non-null  float64
 8   Height          18159 non-null  object 
 9   Weight          18159 non-null  object 
 10  Special         18207 non-null  int64  
 11  Preferred Foot  18159 non-null  object 
 12  Weak Foot       18159 non-null  float64
 13  Acceleration    18159 non-null  float64
 14  SprintSpeed     18159 non-null  float64
 15  Agility         18159 non-null  float64
 16  Balance         18159 non-null  float64
 17  BallControl     18159 non-null 

In [73]:
df.describe()

,Age,Overall,Potential,Value,Wage,Special,Weak Foot,Acceleration,SprintSpeed,Agility,Balance,BallControl,Composure,Dribbling,Reactions,Skill Moves,ShotPower,Jumping,LongShots,Dribbling.1,Penalties,Jersey Number
count,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18147.000000
mean,25.122206,66.238699,71.307299,265.026929,9.731312,1597.809908,2.947299,64.614076,64.726967,63.503607,63.966573,58.369459,58.648274,55.371001,61.836610,2.361308,55.460047,65.089432,47.109973,55.371001,48.548598,19.546096
std,4.669943,6.908930,6.136496,289.564928,21.999290,272.586016,0.660456,14.927780,14.649953,14.766049,14.136166,16.686595,11.436133,18.910371,9.010464,0.756164,17.237958,11.820044,19.260524,18.910371,15.704053,15.947765
min,16.000000,46.000000,48.000000,0.000000,0.000000,731.000000,1.000000,12.000000,12.000000,14.000000,16.000000,5.000000,3.000000,4.000000,21.000000,1.000000,2.000000,15.000000,3.000000,4.000000,5.000000,1.000000
25%,21.000000,62.000000,67.000000,4.400000,1.000000,1457.000000,3.000000,57.000000,57.000000,55.000000,56.000000,54.000000,51.000000,49.000000,56.000000,2.000000,45.000000,58.000000,33.000000,49.000000,39.000000,8.000000
50%,25.000000,66.000000,71.000000,160.000000,3.000000,1635.000000,3.000000,67.000000,67.000000,66.000000,66.000000,63.000000,60.000000,61.000000,62.000000,2.000000,59.000000,66.000000,51.000000,61.000000,49.000000,17.000000
75%,28.000000,71.000000,75.000000,475.000000,9.000000,1787.000000,3.000000,75.000000,75.000000,74.000000,74.000000,69.000000,67.000000,68.000000,68.000000,3.000000,68.000000,73.000000,62.000000,68.000000,60.000000,26.000000
max,45.000000,94.000000,95.000000,975.000000,565.000000,2346.000000,5.000000,97.000000,96.000000,96.000000,96.000000,96.000000,96.000000,97.000000,96.000000,5.000000,95.000000,95.000000,94.000000,97.000000,92.000000,99.000000


### 缺失值？

In [ ]:
df.isnull().any()

In [47]:
# df['Jersey Number'].isnull()
df[df['Jersey Number'].isnull()][['Name', 'Club', 'Nationality', 'Wage', 'Jersey Number']]

,Name,Club,Nationality,Wage,Jersey Number
5018,R. Raldes,NaN,Bolivia,0.0,NaN
6736,J. Arce,NaN,Bolivia,0.0,NaN
7922,L. Gutiérrez,NaN,Bolivia,0.0,NaN
9905,R. Vargas,NaN,Bolivia,0.0,NaN
10628,D. Bejarano,NaN,Bolivia,0.0,NaN
13236,J. McNulty,Rochdale,Scotland,1.0,NaN
13237,J. Barrera,Boyacá Chicó FC,Nicaragua,1.0,NaN
13238,J. Stead,Notts County,England,3.0,NaN
13239,A. Semprini,Brescia,Italy,1.0,NaN
13240,R. Bingham,Hamilton Academical FC,England,1.0,NaN


## 哪個球會最有錢？球員平均薪水最高

In [ ]:
df.pivot_table(index='Club', aggfunc='mean')

In [ ]:
df.pivot_table(index='Club', aggfunc='mean')['Wage'].sort_values(ascending=False)

In [ ]:
df.groupby(['Club'])['Wage']

In [ ]:
df.groupby(['Club'])['Wage'].agg('mean').to_frame().sort_values(by='Wage', ascending=False)

## 穿10號球衣的球員，誰的薪水最高